In [1]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
import env

The dataset for these exercises lives in the Codeup Data Science MySQL Server. The database name is curriculum_logs.

Hint: You will need to explore the database (yay SQL!)

In [2]:
url = f'mysql+pymysql://{env.username}:{env.password}@{env.host}/curriculum_logs'
query = '''
SELECT date,
       path as endpoint,
       user_id,
       ip as source_ip,
       name as cohort,
       program_id,
       start_date,
       end_date
FROM logs
JOIN cohorts ON cohorts.id = logs.cohort_id;
'''
df = pd.read_sql(query, url)
df.head()

,date,endpoint,user_id,source_ip,cohort,program_id,start_date,end_date
0,2018-01-26,/,1,97.105.19.61,Hampton,1,2015-09-22,2016-02-06
1,2018-01-26,java-ii,1,97.105.19.61,Hampton,1,2015-09-22,2016-02-06
2,2018-01-26,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,1,2015-09-22,2016-02-06
3,2018-01-26,slides/object_oriented_programming,1,97.105.19.61,Hampton,1,2015-09-22,2016-02-06
4,2018-01-26,javascript-i/conditionals,2,97.105.19.61,Teddy,2,2018-01-08,2018-05-17


In [3]:
df.date = pd.to_datetime(df.date)
df = df.set_index(df.date)

In [20]:
df.tail()

,date,endpoint,user_id,source_ip,cohort,program_id,start_date,end_date
date,,,,,,,,
2021-04-21,2021-04-21,jquery/personal-site,64,71.150.217.33,Staff,2,2014-02-04,2014-02-04
2021-04-21,2021-04-21,jquery/mapbox-api,64,71.150.217.33,Staff,2,2014-02-04,2014-02-04
2021-04-21,2021-04-21,jquery/ajax/weather-map,64,71.150.217.33,Staff,2,2014-02-04,2014-02-04
2021-04-21,2021-04-21,anomaly-detection/discrete-probabilistic-methods,744,24.160.137.86,Staff,2,2014-02-04,2014-02-04
2021-04-21,2021-04-21,jquery/mapbox-api,64,71.150.217.33,Staff,2,2014-02-04,2014-02-04


1. Label students by the program they are in.

In [5]:
df.program_id.value_counts()

2    713365
3    103412
1     30548
4         5
Name: program_id, dtype: int64

2. Is it possible to identify the user_id of a staff member?

In [6]:
staff = df[df.cohort == 'Staff']

In [7]:
staff.user_id.unique()

array([ 53, 314,  40,  64,  11, 211,   1, 312, 146, 248, 370, 397, 404,
       257, 428, 461,  37, 514, 539, 545, 546, 572, 315,  41, 592, 618,
       620, 521, 652, 502, 653, 480, 738, 742, 745, 813, 430, 816, 581,
       854, 855, 744, 893, 148, 894, 513, 630, 308, 951, 953, 980])

3. Identify students who are accessing our curriculum pages beyond the end of their time at Codeup.

In [8]:
df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date'] = pd.to_datetime(df['end_date'])

In [11]:
df[df.date > df.end_date].user_id.unique()

array([  1,  11,  21,  26,  37,  40,  50,  51,  53,  60,  64,  66,  67,
        68,  69,  70,  71,  72,  75,  76,  77,  80,  81,  82,  83,  84,
        85,  87,  90,  91,  92,  93,  94,  95,  96,  97,  98, 101, 102,
        16,  42,  41,  17,  22,  15, 104,  47,  57,  10, 105, 106,  28,
       107,  65,  52,  33,  56, 108,  27,  38,  36, 109, 110, 112, 113,
       114, 115, 139, 140, 141, 143, 144, 146, 147, 148, 149, 150, 151,
       152, 153, 156, 161, 162, 163, 164, 165, 167, 168, 169, 170, 172,
       173, 174, 175, 177, 178, 179,  29,  19, 180,   6,  99,  43,  31,
         7, 181, 138,  35, 142,   5, 155, 154,  88, 209, 210,  18,   3,
        20, 118, 211, 212, 214, 216, 217, 131, 135, 128, 127, 124, 130,
       159,  12, 145, 241, 242, 136, 157, 171, 123,  25, 134, 119, 243,
       244, 245, 246, 160, 248, 252, 254, 255, 120, 257, 279, 280, 281,
        14, 253, 188, 186, 283, 184, 215, 278, 183, 204, 193, 195, 197,
       249, 206, 284, 121, 201, 285, 286, 311, 190, 312, 116, 20

4. Identify students who present anomalous activity using the Bollinger Band method, but reduce K to 2.

In [12]:
def one_user_df_prep(df, user):
    '''
    This function returns a dataframe consisting of data for only a single defined user
    '''
    df = df[df.user_id == user]
    df.date = pd.to_datetime(df.date)
    df = df.set_index(df.date)
    pages_one_user = df['endpoint'].resample('d').count()
    return pages_one_user

In [13]:
def compute_pct_b(pages_one_user, span, weight, user):
    '''
    This function adds the %b of a bollinger band range for the page views of a single user's log activity
    '''
    # Calculate upper and lower bollinger band
    midband = pages_one_user.ewm(span=span).mean()
    stdev = pages_one_user.ewm(span=span).std()
    ub = midband + stdev*weight
    lb = midband - stdev*weight
    
    # Add upper and lower band values to dataframe
    bb = pd.concat([ub, lb], axis=1)
    
    # Combine all data into a single dataframe
    my_df = pd.concat([pages_one_user, midband, bb], axis=1)
    my_df.columns = ['pages_one_user', 'midband', 'ub', 'lb']
    
    # Calculate percent b and relevant user id to dataframe
    my_df['pct_b'] = (my_df['pages_one_user'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])
    my_df['user_id'] = user
    return my_df

In [14]:
def find_anomalies(df, user, span, weight, plot=False):
    '''
    This function returns the records where a user's daily activity exceeded the upper limit of a bollinger band range
    '''
    
    # Reduce dataframe to represent a single user
    pages_one_user = one_user_df_prep(df, user)
    
    # Add bollinger band data to dataframe
    my_df = compute_pct_b(pages_one_user, span, weight, user)
    
    # Plot data if requested (plot=True)
    if plot:
        plot_bands(my_df, user)
    
    # Return only records that sit outside of bollinger band upper limit
    return my_df[my_df.pct_b>1]

In [15]:
span = 30
weight = 2

anomalies = pd.DataFrame()
for u in list(df.user_id.unique()):
    user_df = find_anomalies(df, u, span, weight)
    anomalies = pd.concat([anomalies, user_df], axis=0)

In [22]:
anomalies.groupby('user_id').pages_one_user.sum()

user_id
1      1529
2       119
3       125
4        84
5       222
       ... 
945      26
947      18
948     119
949      40
951     122
Name: pages_one_user, Length: 767, dtype: int64

In [18]:
anomalies.sort_values(by='pages_one_user', ascending=False)

,pages_one_user,midband,ub,lb,pct_b,user_id
date,,,,,,
2020-07-22,285,44.915959,192.499064,-102.667146,1.313386,11
2019-03-03,272,24.721632,163.040772,-113.597508,1.393869,341
2020-04-16,271,44.194413,193.288101,-104.899276,1.260614,570
2019-07-25,253,34.698961,160.039466,-90.641543,1.370832,422
2020-04-26,234,54.999406,211.467475,-101.468663,1.072004,570
...,...,...,...,...,...,...
2019-01-15,1,0.096889,0.790202,-0.596425,1.151301,50
2019-03-19,1,0.066996,0.668360,-0.534368,1.275740,219
2019-01-19,1,0.138718,0.903715,-0.626278,1.062932,50


5. Plot the access activity of these students.

In [17]:
def plot_bands(my_df, user):
    '''
    This function plots the bolliger bands of the page views for a single user
    '''
    fig, ax = plt.subplots(figsize=(12,8))
    ax.plot(my_df.index, my_df.pages_one_user, label='Number of Pages, User: '+str(user))
    ax.plot(my_df.index, my_df.midband, label = 'EMA/midband')
    ax.plot(my_df.index, my_df.ub, label = 'Upper Band')
    ax.plot(my_df.index, my_df.lb, label = 'Lower Band')
    ax.legend(loc='best')
    ax.set_ylabel('Number of Pages')
    plt.show()

**BONUS:** Can you identify users who are viewing both the web dev and data science curriculum?